# Day 0

Import commonly used modules.

In [1]:
using DelimitedFiles
using Base.Iterators
using Combinatorics

# Day 1

## Part 1

After feeling like you've been falling for a few minutes, you look at the device's tiny screen. "Error: Device must be calibrated before first use. Frequency drift detected. Cannot maintain destination lock." Below the message, the device shows a sequence of changes in frequency (your puzzle input). A value like +6 means the current frequency increases by 6; a value like -3 means the current frequency decreases by 3.

For example, if the device displays frequency changes of +1, -2, +3, +1, then starting from a frequency of zero, the following changes would occur:

Current frequency  0, change of +1; resulting frequency  1.
Current frequency  1, change of -2; resulting frequency -1.
Current frequency -1, change of +3; resulting frequency  2.
Current frequency  2, change of +1; resulting frequency  3.
In this example, the resulting frequency is 3.

Here are other example situations:

+1, +1, +1 results in  3
+1, +1, -2 results in  0
-1, -2, -3 results in -6
Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?

Inputs saved in day1/input1.txt

In [2]:
open("day1input.txt") do f
    global d1inputs = readdlm(f, Int64)
end

pos = sum(d1inputs)
print("Final resulting frequency: $pos")

Final resulting frequency: 595

## Part Two

You notice that the device repeats the same frequency change list over and over. To calibrate the device, you need to find the first frequency it reaches twice.

For example, using the same list of changes above, the device would loop as follows:

Current frequency  0, change of +1; resulting frequency  1.
Current frequency  1, change of -2; resulting frequency -1.
Current frequency -1, change of +3; resulting frequency  2.
Current frequency  2, change of +1; resulting frequency  3.
(At this point, the device continues from the start of the list.)
Current frequency  3, change of +1; resulting frequency  4.
Current frequency  4, change of -2; resulting frequency  2, which has already been seen.
In this example, the first frequency reached twice is 2. Note that your device might need to repeat its list of frequency changes many times before a duplicate frequency is found, and that duplicates might be found while in the middle of processing the list.

Here are other examples:

+1, -1 first reaches 0 twice.
+3, +3, +4, -2, -4 first reaches 10 twice.
-6, +3, +8, +5, -6 first reaches 5 twice.
+7, +7, -2, -7, -4 first reaches 14 twice.

What is the first frequency your device reaches twice?

In [3]:
using Base.Iterators

visited = Set{Int64}()
pos = 0
push!(visited, pos)

for input in cycle(d1inputs)
    pos += input
    if in(pos, visited)
        break
    end
    push!(visited, pos)
end

print("First frequency reached twice is: $pos")

First frequency reached twice is: 80598

# Day 2: Inventory Management System

You stop falling through time, catch your breath, and check the screen on the device. "Destination reached. Current Year: 1518. Current Location: North Pole Utility Closet 83N10." You made it! Now, to find those anomalies.

Outside the utility closet, you hear footsteps and a voice. "...I'm not sure either. But now that so many people have chimneys, maybe he could sneak in that way?" Another voice responds, "Actually, we've been working on a new kind of suit that would let him fit through tight spaces like that. But, I heard that a few days ago, they lost the prototype fabric, the design plans, everything! Nobody on the team can even seem to remember important details of the project!"

"Wouldn't they have had enough fabric to fill several boxes in the warehouse? They'd be stored together, so the box IDs should be similar. Too bad it would take forever to search the warehouse for two similar box IDs..." They walk too far away to hear any more.

Late at night, you sneak to the warehouse - who knows what kinds of paradoxes you could cause if you were discovered - and use your fancy wrist device to quickly scan every box and produce a list of the likely candidates (your puzzle input).

To make sure you didn't miss any, you scan the likely candidate boxes again, counting the number that have an ID containing exactly two of any letter and then separately counting those with exactly three of any letter. You can multiply those two counts together to get a rudimentary checksum and compare it to what your device predicts.

For example, if you see the following box IDs:

- `abcdef` contains no letters that appear exactly two or three times.
- `bababc` contains two a and three b, so it counts for both.
- `abbcde` contains two b, but no letter appears exactly three times.
- `abcccd` contains three c, but no letter appears exactly two times.
- `aabcdd` contains two a and two d, but it only counts once.
- `abcdee` contains two e.
- `ababab` contains three a and three b, but it only counts once.

Of these box IDs, four of them contain a letter which appears exactly twice, and three of them contain a letter which appears exactly three times. Multiplying these together produces a checksum of 4 * 3 = 12.

What is the checksum for your list of box IDs?

In [4]:
function getLetterCounts(s)
    counts = Dict()
    for c in s
        counts[c] = get(counts, c, 0) + 1
    end
    return counts
end

# Example function input (remove ';' to show output)
getLetterCounts("bababc");

In [5]:
function numCodesWithNRepeats(n, inputs)
    letters = Char[]
    num = 0
    for code in inputs
        counts = getLetterCounts(code)
        if n in values(counts)
            num +=1
        end
    end
    return num
end

# Example function input (remove ';' to show output)
numCodesWithNRepeats(2, ["abbcde", "bababc", "abcdef"]);

In [6]:
open("day2input.txt") do f
    global d2inputs = readdlm(f, String)
end

function checksum(inputs)
    repeat2 = numCodesWithNRepeats(2, inputs)
    repeat3 = numCodesWithNRepeats(3, inputs)
    return repeat2 * repeat3
end

checksum(d2inputs)

6370

## Part Two

Confident that your list of box IDs is complete, you're ready to find the boxes full of prototype fabric.

The boxes will have IDs which differ by exactly one character at the same position in both strings. For example, given the following box IDs:

abcde
fghij
klmno
pqrst
fguij
axcye
wvxyz
The IDs abcde and axcye are close, but they differ by two characters (the second and fourth). However, the IDs fghij and fguij differ by exactly one character, the third (h and u). Those must be the correct boxes.

What letters are common between the two correct box IDs? (In the example above, this is found by removing the differing character from either ID, producing fgij.)

In [7]:
function numChanges(a, b)
    n = 0
    for (c1, c2) in zip(a, b)
        if c1 != c2
            n +=1
        end
    end
    return n
end

numChanges("fghij", "fguij");

In [8]:
function pairsWithOneChange(inputs)
    pairs = []
    for (s1, s2) in combinations(inputs, 2)
        if numChanges(s1, s2) == 1
            push!(pairs, (s1, s2))
        end
    end
    return pairs
end

pairsWithOneChange(["abc", "acd", "adc", "aec"]);

In [9]:
function commonLetters(s1, s2)
    output = []
    for (c1, c2) in zip(s1, s2)
        if c1 == c2
            push!(output, c1)
        end
    end
    return join(output)
end

commonLetters("123456789", "123356789");

In [10]:
function commonOfCorrect(inputs)
    oneChange = pairsWithOneChange(inputs)
    commonLetters(oneChange[1]...)
end

commonOfCorrect(d2inputs)

"rmyxgdlihczskunpfijqcebtv"

# Day 3: No Matter How You Slice It

The Elves managed to locate the chimney-squeeze prototype fabric for Santa's suit (thanks to someone who helpfully wrote its box IDs on the wall of the warehouse in the middle of the night). Unfortunately, anomalies are still affecting them - nobody can even agree on how to cut the fabric.

The whole piece of fabric they're working on is a very large square - at least 1000 inches on each side.

Each Elf has made a claim about which area of fabric would be ideal for Santa's suit. All claims have an ID and consist of a single rectangle with edges parallel to the edges of the fabric. Each claim's rectangle is defined as follows:

The number of inches between the left edge of the fabric and the left edge of the rectangle.
The number of inches between the top edge of the fabric and the top edge of the rectangle.
The width of the rectangle in inches.
The height of the rectangle in inches.
A claim like `#123 @ 3,2: 5x4` means that claim ID 123 specifies a rectangle 3 inches from the left edge, 2 inches from the top edge, 5 inches wide, and 4 inches tall. Visually, it claims the square inches of fabric represented by # (and ignores the square inches of fabric represented by .) in the diagram below:

```...........
...........
...#####...
...#####...
...#####...
...#####...
...........
...........
...........```

The problem is that many of the claims overlap, causing two or more claims to cover part of the same areas. For example, consider the following claims:

`#1 @ 1,3: 4x4`

`#2 @ 3,1: 4x4`

`#3 @ 5,5: 2x2`

Visually, these claim the following areas:

```........
...2222.
...2222.
.11XX22.
.11XX22.
.111133.
.111133.
........```

The four square inches marked with X are claimed by both 1 and 2. (Claim 3, while adjacent to the others, does not overlap either of them.)

If the Elves all proceed with their own plans, none of them will have enough fabric. How many square inches of fabric are within two or more claims?

In [11]:
function parseClaim(s)
    split(s, r"[^0-9]") |>
    parts -> filter(!isempty, parts) |>
    nums -> map(n -> parse(Int64, n), nums)
end

parseClaim (generic function with 1 method)

In [12]:
function getAffectedSquaresIterator(_, x, y, w, h)
    Iterators.product(x:x+w-1, y:y+h-1)
end
getAffectedSquares(parseClaim("#1 @ 1,3: 4x4")...)

UndefVarError: UndefVarError: getAffectedSquares not defined

In [13]:
function getInput()
    open("day3input.txt") do f
        readlines(f) |>
        collect |>
        lines -> map(line -> parseClaim(line), lines)
    end
end;

In [14]:
function addPointToDict!(dict, xy)
    dict[(xy[1], xy[2])] = get(dict, (xy[1], xy[2]), 0) + 1
end

function addClaimToDict!(dict, claim)
    getAffectedSquaresIterator(claim...) |>
    iter -> foreach(xy -> addPointToDict!(dict, xy), iter)
end
dict = Dict()
addClaimToDict!(dict, (1, 1, 3, 4, 4))
dict

Dict{Any,Any} with 16 entries:
  (4, 3) => 1
  (1, 6) => 1
  (3, 6) => 1
  (2, 3) => 1
  (2, 6) => 1
  (4, 4) => 1
  (4, 6) => 1
  (2, 5) => 1
  (1, 4) => 1
  (3, 3) => 1
  (1, 3) => 1
  (4, 5) => 1
  (2, 4) => 1
  (1, 5) => 1
  (3, 5) => 1
  (3, 4) => 1

In [15]:
function part1()
    claims = getInput()
    coverage = Dict()
    for claim in claims
        addClaimToDict!(coverage, claim)
    end
    overlaps = 0
    for (k, v) in coverage
        if v > 1
            overlaps += 1
        end
    end
    return overlaps
end
print("There are " * string(part1()) * " square inches of overlap.")

There are 101469 square inches of overlap.

In [16]:
function noOverlappingSquares(dict, claim)
    affectedSquares = getAffectedSquares(claim...)
    for square in affectedSquares
        if dict[square] > 1
            return false
        end
    end
    return true
end
        
function part2()
    claims = getInput()
    overlappingSquareDict = Dict()
    for claim in claims
        addClaimToDict!(overlappingSquareDict, claim)
    end
    
    filter(claim -> noOverlappingSquares(overlappingSquareDict, claim), claims)[1][1]
end
print("Claim #" * string(part2()) * " has no overlapping squares.")

UndefVarError: UndefVarError: getAffectedSquares not defined

# Day 4: Repose Record

You've sneaked into another supply closet - this time, it's across from the prototype suit manufacturing lab. You need to sneak inside and fix the issues with the suit, but there's a guard stationed outside the lab, so this is as close as you can safely get.

As you search the closet for anything that might help, you discover that you're not the first person to want to sneak in. Covering the walls, someone has spent an hour starting every midnight for the past few months secretly observing this guard post! They've been writing down the ID of the one guard on duty that night - the Elves seem to have decided that one guard was enough for the overnight shift - as well as when they fall asleep or wake up while at their post (your puzzle input).

For example, consider the following records, which have already been organized into chronological order:

```
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
```

Timestamps are written using year-month-day hour:minute format. The guard falling asleep or waking up is always the one whose shift most recently started. Because all asleep/awake times are during the midnight hour (00:00 - 00:59), only the minute portion (00 - 59) is relevant for those events.

Visually, these records show that the guards are asleep at these times:

```
Date   ID   Minute
            000000000011111111112222222222333333333344444444445555555555
            012345678901234567890123456789012345678901234567890123456789
11-01  #10  .....####################.....#########################.....
11-02  #99  ........................................##########..........
11-03  #10  ........................#####...............................
11-04  #99  ....................................##########..............
11-05  #99  .............................................##########.....
```

The columns are Date, which shows the month-day portion of the relevant day; ID, which shows the guard on duty that day; and Minute, which shows the minutes during which the guard was asleep within the midnight hour. (The Minute column's header shows the minute's ten's digit in the first row and the one's digit in the second row.) Awake is shown as ., and asleep is shown as #.

Note that guards count as asleep on the minute they fall asleep, and they count as awake on the minute they wake up. For example, because Guard #10 wakes up at 00:25 on 1518-11-01, minute 25 is marked as awake.

If you can figure out the guard most likely to be asleep at a specific time, you might be able to trick that guard into working tonight so you can have the best chance of sneaking in. You have two strategies for choosing the best guard/minute combination.

Strategy 1: Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

In the example above, Guard #10 spent the most minutes asleep, a total of 50 minutes (20+25+5), while Guard #99 only slept for a total of 30 minutes (10+10+10). Guard #10 was asleep most during minute 24 (on two days, whereas any other minute the guard was asleep was only seen on one day).

While this example listed the entries in chronological order, your entries are in the order you found them. You'll need to organize them before they can be analyzed.

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 10 * 24 = 240.)

In [17]:
function getSortedInput()
    inputs = Array{String}(undef, 0)
    open("day4input.txt") do f
        for line in eachline(f)
            push!(inputs, string(line))
        end
    end
    return sort(inputs)
end;
getSortedInput();

In [18]:
function extractMinute(line)
    return parse(Int64, line[16:17])
end

function extractGuardId(lines)
    split(lines[1])[4][2:end]
end

extractMinute("[1518-11-21 23:50] Guard #2953 begins shift")
# extractGuardId(["[1518-11-21 23:50] Guard #2953 begins shift"])

50

In [19]:
function partitionByGuard(sortedInput)
    guards = []
    guard = []
    for line in sortedInput
        if occursin("Guard", line) && length(guard) > 0
            push!(guards, guard)
            guard = []
        end
        push!(guard, line)
    end
    
    dict = Dict()
    for guard in guards
        id = extractGuardId(guard)
        dict[id] = append!(get(dict, id, []), guard)
    end
    return collect(values(dict))
end

partitionByGuard(getSortedInput())

23-element Array{Any,1}:
 Any["[1518-10-31 00:00] Guard #683 begins shift"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
 Any["[1518-03-27 00:01] Guard #647 begins shift", "[1518-03-27 00:08] falls asleep", "[1518-03-27 00:12] wakes up", "[1518-03-27 00:22] falls asleep", "[1518-03-27 00:35] wakes up", "[1518-04-22 00:00] Guard #647 begins sh

In [20]:
function lineContainsSleepOrWakeTime(line)
    occursin(r"asleep|wakes", line)
end

lineContainsSleepOrWakeTime (generic function with 1 method)

In [21]:
function sumTimeAsleepForGuard(lines)
    id = extractGuardId(lines)
    sleepAndWakeMinutes = map(extractMinute, filter(lineContainsSleepOrWakeTime, lines))
    minutesAsleep = Iterators.partition(sleepAndWakeMinutes, 2) |>
        iter -> map(xy -> xy[2] - xy[1], iter) |>
        iter -> reduce(+, iter)
    return (id, minutesAsleep)
end

sumTimeAsleepForGuard(["[1518-11-01 00:00] Guard #10 begins shift";
"[1518-11-01 00:05] falls asleep";
"[1518-11-01 00:25] wakes up";
"[1518-11-01 00:30] falls asleep";
"[1518-11-01 00:55] wakes up"])

("10", 45)

In [22]:
function findGuardAsleepMost(inputs)
    guards = partitionByGuard(inputs)
    guardIdToTimeAsleep = map(sumTimeAsleepForGuard, guards)
    max = 0
    maxId = "N/A"
    for (id, timeAsleep) in guardIdToTimeAsleep
        if timeAsleep > max
            max = timeAsleep
            maxId = id
        end
    end
    return (maxId, max)
end
findGuardAsleepMost(getSortedInput())

("2953", 465)

In [23]:
function getGuardLinesForId(id, inputs)
    guards = partitionByGuard(inputs)
    for guard in guards
        if occursin("Guard #$id", guard[1])
            return guard
        end
    end
end
getGuardLinesForId("2953", getSortedInput())

67-element Array{Any,1}:
 "[1518-03-19 00:02] Guard #2953 begins shift"
 "[1518-03-19 00:37] falls asleep"            
 "[1518-03-19 00:41] wakes up"                
 "[1518-04-08 23:57] Guard #2953 begins shift"
 "[1518-04-09 00:31] falls asleep"            
 "[1518-04-09 00:47] wakes up"                
 "[1518-05-04 23:56] Guard #2953 begins shift"
 "[1518-05-05 00:07] falls asleep"            
 "[1518-05-05 00:17] wakes up"                
 "[1518-05-05 00:23] falls asleep"            
 "[1518-05-05 00:52] wakes up"                
 "[1518-05-08 23:59] Guard #2953 begins shift"
 "[1518-05-09 00:25] falls asleep"            
 ⋮                                            
 "[1518-10-07 00:55] wakes up"                
 "[1518-11-17 00:02] Guard #2953 begins shift"
 "[1518-11-17 00:22] falls asleep"            
 "[1518-11-17 00:41] wakes up"                
 "[1518-11-18 23:58] Guard #2953 begins shift"
 "[1518-11-19 00:12] falls asleep"            
 "[1518-11-19 00:59] wakes up"     

In [24]:
function findMinuteGuardIsAsleepMost(lines)
    sleepAndWakeMinutes = map(extractMinute, filter(lineContainsSleepOrWakeTime, lines))
    daysAsleepByMinute = Dict()
    for (asleep, awake) in Iterators.partition(sleepAndWakeMinutes, 2)
        for m in asleep:awake-1
            daysAsleepByMinute[m] = get(daysAsleepByMinute, m, 0) + 1
        end
    end
    max = 0
    maxMinute = -1
    for (minute, timesAsleep) in daysAsleepByMinute
        if timesAsleep > max
            max = timesAsleep
            maxMinute = minute
        end
    end
    return (max, maxMinute)
end

function solvePart1()
    inputs = getSortedInput()
    (id, minutes) = findGuardAsleepMost(inputs)
    guard = getGuardLinesForId(id, inputs)
    (max, minute) = findMinuteGuardIsAsleepMost(guard)
    return (id, minute)
end
solvePart1()

("2953", 39)

## Part Two

Strategy 2: Of all guards, which guard is most frequently asleep on the same minute?

In the example above, Guard #99 spent minute 45 asleep more than any other guard or minute - three times in total. (In all other cases, any guard spent any minute asleep at most twice.)

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 99 * 45 = 4455.)

In [25]:
function solvePart2()
    inputs = getSortedInput()
    guards = partitionByGuard(inputs)
    max = 0
    maxMinute = -1
    maxId = "n/a"
#     guardToMaxMinute = Dict()
#     for guard in guards
#         guardToMaxMinute[extractGuardId(guard)] = findMinuteGuardIsAsleepMost(guard)
#     end
#     return guardToMaxMinute
    for guard in guards
        (timesAsleep, minute) = findMinuteGuardIsAsleepMost(guard)
        if timesAsleep >= max
            max = timesAsleep
            maxMinute = minute
            maxId = extractGuardId(guard)
        end
    end
    return (maxId, maxMinute, max)
end
solvePart2()

("1069", 30, 17)